In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
import os
import torch
from tqdm import tqdm
from pyannote.audio import Pipeline

pretrained_pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=True
)
pretrained_pipeline.to(torch.device("cuda"))

/data/saiakarsh/envs/augnito/lib/python3.9/site-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


'Primock.SpeakerDiarization.full' found in /data/saiakarsh/codes/diar_aug/database.yml does not define the 'scope' of speaker labels (file, database, or global). Setting it to 'file'.


/data/saiakarsh/envs/augnito/lib/python3.9/site-packages/torch_audiomentations/utils/io.py:27: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
torchvision is not available - cannot save figures


In [22]:
out_dir = "timestamps_pyan_diar_pretrained"
os.mkdir(out_dir)

src_dir = "primock57/output/mixed_audio/"
for audio in tqdm(os.listdir(src_dir)):
    audio_path = os.path.join(src_dir, audio)
    diarization = pretrained_pipeline(audio_path)
    meta = []
    for seg, seg_id, spk_id in diarization.itertracks(yield_label=True):
        if len(meta) > 0 and meta[-1][2] == spk_id: # last spk_id == spk_id
            meta[-1][1] = seg.end
        else:
            meta.append([seg.start, seg.end, spk_id])
    with open(os.path.join(out_dir, audio[:-3] + "txt"), 'w') as f:
        for st, en, spk_id in meta:
            f.write(f"{st:.3f},{en:.3f},{spk_id}\n")

100%|██████████| 57/57 [51:25<00:00, 54.14s/it] 


In [6]:
from pyannote.audio.pipelines import SpeakerDiarization

finetuned_model = "/data/saiakarsh/codes/diar_aug/lightning_logs/version_0/checkpoints/epoch=19.ckpt"
best_segmentation_threshold = 0.5952774382125149
best_clustering_threshold = 0.7995650252252342

finetuned_pipeline = SpeakerDiarization(
    segmentation=finetuned_model,
    embedding=pretrained_pipeline.embedding,
    embedding_exclude_overlap=pretrained_pipeline.embedding_exclude_overlap,
    clustering=pretrained_pipeline.klustering,
)

finetuned_pipeline.instantiate({
    "segmentation": {
        "threshold": best_segmentation_threshold,
        "min_duration_off": 0.0,
    },
    "clustering": {
        "method": "centroid",
        "min_cluster_size": 15,
        "threshold": best_clustering_threshold,
    },
})

finetuned_pipeline.to(torch.device("cuda"))

In [7]:
out_dir = "timestamps_pyan_diar_finetuned"
os.mkdir(out_dir)

src_dir = "primock57/output/mixed_audio/"
for audio in tqdm(os.listdir(src_dir)):
    audio_path = os.path.join(src_dir, audio)
    diarization = finetuned_pipeline(audio_path)
    meta = []
    for seg, seg_id, spk_id in diarization.itertracks(yield_label=True):
        if len(meta) > 0 and meta[-1][2] == spk_id: # last spk_id == spk_id
            meta[-1][1] = seg.end # combine timestamps if last spk_id is same as current one
        else:
            meta.append([seg.start, seg.end, spk_id])
    with open(os.path.join(out_dir, audio[:-3] + "txt"), 'w') as f:
        for st, en, spk_id in meta:
            f.write(f"{st:.3f},{en:.3f},{spk_id}\n")

100%|██████████| 57/57 [1:26:24<00:00, 90.95s/it] 
